# TECHNICAL REPORT: VRP with Increased Capacity (80 kg)

## 1. Introduction and Scenario Change

This experiment addresses the **Vehicle Routing Problem with Capacity (CVRP)** under a new operational constraint. Unlike the previous scenario, the fleet capacity has been increased from 60 kg to **80 kg**.

**Experiment Objective:**
To analyze how the capacity increase affects **customer feasibility** and **route efficiency**. Specifically, we seek to integrate the customer with the highest demand (C5), who was previously excluded, and minimize the total distance traveled while serving 100% of the customer portfolio.

## 2. Data Analysis and Constraints

### 2.1. Demand Profile
The calculated demands for the 5 customers are:

| Customer | Demand (kg) | Status (Cap. 60) | Status (Cap. 80) |
| :---: | :---: | :---: | :---: |
| C1 | 18.6 | Valid | **Valid** |
| C2 | 22.8 | Valid | **Valid** |
| C3 | 48.0 | Valid | **Valid** |
| C4 | 60.0 | Valid | **Valid** |
| C5 | **77.7** | *Invalid* | **Valid** |

### 2.2. Impact of New Capacity
The shift to 80 kg has a critical impact:
1.  **Total Inclusion:** Customer 5 (77.7 kg) now meets the constraint ($77.7 \le 80$). The Genetic Algorithm (GA) must now optimize a route for **5 customers** instead of 4.
2.  **Load Saturation:** Customer 5 will occupy 97% of a truck's capacity, practically forcing an exclusive trip for them.
3.  **Combinatorics:** The extra capacity allows for new groupings. For example, C4 (60 kg) now leaves 20 kg free, allowing it to be grouped with C1 (18.6 kg), something impossible with the 60 kg capacity.

## 3. Methodology (Genetic Algorithm)

The GA configuration is maintained to ensure comparability:
* **Representation:** Permutation of indices of all valid customers.
* **Evaluation (Fitness):** Total Euclidean distance.
* **Route Decoding:** "First Fit" Heuristic (fill the vehicle until the next customer does not fit, return to depot).
* **Parameters:** 200 Generations, Population of 50 individuals.

## 4. VRP Code Implementation (Capacity 80)
Below are the code cells for the experiment execution.

In [1]:
import random
import math
import matplotlib.pyplot as plt

# Depot coordinates
depot = [20, 120]

# Customer coordinates
customers = [
    [35, 115], [50, 140], [70, 100], [40, 80], [25, 60]
]

# Product weights (kg)
weights = [1.2, 3.8, 7.5, 0.9, 15.4, 12.1, 4.3, 19.7, 8.6, 2.5]

# Customer orders: (product_id, quantity)
orders = [
    [(3, 2), (1, 3)],
    [(2, 6)],
    [(7, 4), (5, 2)],
    [(3, 8)],
    [(6, 5), (9, 2)]
]

# PARAMS
N_GEN = 200
POP_SIZE = 50
PM = 0.1

In [2]:
# Demand Calculation
demands = []
for order in orders:
    # Calculate total weight for the order based on product weights
    total = sum(weights[item - 1] * qty for item, qty in order)
    demands.append(total)

# --- CRITICAL CHANGE: Capacity increased to 80 ---
capacity = 80

print('Calculated demands:', [round(x, 1) for x in demands])
print(f'Vehicle Capacity: {capacity} kg')

Calculated demands: [18.6, 22.8, 48.0, 60.0, 77.7]
Vehicle Capacity: 80 kg


In [3]:
# Classification of Valid / Invalid Customers
valid_customers = []
valid_demands = []
invalid_customers = []
invalid_demands = []

for i in range(len(customers)):
    if demands[i] <= capacity:
        # Save valid customer. The index corresponds to the position in the valid list.
        valid_customers.append(customers[i])
        valid_demands.append(demands[i])
    else:
        # This list should remain empty in this scenario (Cap 80)
        invalid_customers.append(customers[i])
        invalid_demands.append(demands[i])

print('Valid customers:', len(valid_customers))
print('Valid demands:', [round(d, 1) for d in valid_demands])
print('Invalid customers:', len(invalid_customers))

Valid customers: 5
Valid demands: [18.6, 22.8, 48.0, 60.0, 77.7]
Invalid customers: 0


In [4]:
# --- Auxiliary & Genetic Functions ---

def distance(a, b):
    """Calculates Euclidean distance between two points."""
    return math.sqrt((a[0] - b[0])**2 + (a[1] - b[1])**2)

def split_into_routes(route):
    """Divides the global sequence (chromosome) into valid routes respecting the 80kg capacity."""
    routes = []
    load = 0
    current_route = []
    for c in route:
        # c is the index of the valid customer
        if load + valid_demands[c] <= capacity:
            current_route.append(c)
            load += valid_demands[c]
        else:
            # Capacity exceeded: current route ends, new one begins.
            routes.append(current_route)
            current_route = [c]
            load = valid_demands[c]
    if current_route:
        # Add the last route if it exists
        routes.append(current_route)
    return routes

def fitness(route):
    """Calculates total distance of all trips generated from the route."""
    routes = split_into_routes(route)
    total = 0
    for r in routes:
        pos = depot # Start at depot
        for c in r:
            # Distance from previous point to customer c
            total += distance(pos, valid_customers[c])
            pos = valid_customers[c]
        # Return distance to depot
        total += distance(pos, depot)
    return total

def create_population(n, num_customers):
    """Creates an initial population of n random routes."""
    population = []
    base = list(range(num_customers))
    for _ in range(n):
        r = base[:]
        random.shuffle(r)
        population.append(r)
    return population

def tournament_selection(population, fitness_values):
    """Selection by binary tournament (chooses the individual with lower fitness)."""
    i1, i2 = random.sample(range(len(population)), 2)
    return population[i1][:] if fitness_values[i1] < fitness_values[i2] else population[i2][:]

def crossover(p1, p2):
    """Order Crossover (OX) modified to maintain chromosome validity."""
    a, b = sorted(random.sample(range(len(p1)), 2))
    child = [-1] * len(p1)
    child[a:b] = p1[a:b]
    pos = b
    for x in p2:
        if x not in child:
            if pos >= len(p1): pos = 0
            child[pos] = x
            pos += 1
    return child

def mutation(route, prob=PM):
    """Mutation by swap with a given probability."""
    if random.random() < prob:
        i, j = random.sample(range(len(route)), 2)
        route[i], route[j] = route[j], route[i]

def genetic_algorithm(n_generations=N_GEN, pop_size=POP_SIZE):
    """Main loop of the Genetic Algorithm."""
    num_customers = len(valid_customers)
    population = create_population(pop_size, num_customers)
    fitness_values = [fitness(r) for r in population]

    for gen in range(n_generations):
        new_population = []
        for _ in range(pop_size):
            p1 = tournament_selection(population, fitness_values)
            p2 = tournament_selection(population, fitness_values)
            child = crossover(p1, p2)
            mutation(child)
            new_population.append(child)
        population = new_population
        fitness_values = [fitness(r) for r in population]

    best_idx = min(range(pop_size), key=lambda i: fitness_values[i])
    return population[best_idx], fitness_values[best_idx]

print("GA Functions ready.")

GA Functions ready.


In [5]:
# Execution and Results
best_route, best_value = genetic_algorithm()

print("Visit Order (Indices of valid customers):")
print(best_route)
print("Total Distance:", round(best_value, 2))

print("\nTrip Details:")
routes = split_into_routes(best_route)
for i, r in enumerate(routes):
    load = sum(valid_demands[c] for c in r)
    print(f"Trip {i+1}: {r} (Load {round(load, 1)} kg)")

Visit Order (Indices of valid customers):
[4, 2, 1, 0, 3]
Total Distance: 350.93

Trip Details:
Trip 1: [4] (Load 77.7 kg)
Trip 2: [2, 1] (Load 70.8 kg)
Trip 3: [0, 3] (Load 78.6 kg)


In [6]:
def plot_routes(routes):
    """Generates the plot of routes, depot, and customers."""
    plt.figure(figsize=(8, 8))
    # Draw customers
    for i, (x, y) in enumerate(valid_customers):
        plt.scatter(x, y, c='blue', zorder=5)
        plt.text(x+1, y+1, f'C{i+1} ({round(valid_demands[i], 1)}kg)', fontsize=9)

    # Draw depot
    plt.scatter(depot[0], depot[1], c='red', s=100, marker='s', label='Depot', zorder=5)

    colors = ['green', 'orange', 'purple', 'cyan']
    
    # Draw routes
    for i, r in enumerate(routes):
        points = [depot] + [valid_customers[c] for c in r] + [depot]
        xs = [p[0] for p in points]
        ys = [p[1] for p in points]
        load = sum(valid_demands[c] for c in r)
        plt.plot(xs, ys, '-o', color=colors[i % len(colors)], label=f'Trip {i+1} ({round(load, 1)}kg)')

    plt.title(f"Optimal Routes (Capacity {capacity}kg)")
    plt.xlabel("X Coordinate")
    plt.ylabel("Y Coordinate")
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.show()

if best_route:
    # We must recalculate routes from the best sequence to plot them
    final_routes = split_into_routes(best_route)
    plot_routes(final_routes)
else:
    print("No valid customers to plot.")

## 5. Final Experiment Conclusions (Capacity 80 kg)

The increase in vehicle capacity from 60 kg to **80 kg** has radically transformed the logistical solution. The main conclusions are:

1.  **Total Viability (C5 Included):**
    * The most critical factor is the inclusion of **Customer 5** (77.7 kg). By increasing capacity, this customer, which previously represented a lost sale or a logistical exception, now falls within standard operational parameters.
    
2.  **Load Optimization (High Filling Rate):**
    * A much more efficient **fleet saturation** is observed.
    * The combination of **Customer 4** (60 kg) with **Customer 1** (18.6 kg) totals **78.6 kg**. This represents an occupancy of **98.2%**, an ideal scenario in logistics that was mathematically impossible with the 60 kg constraint.
    * Customer 5 occupies **97.1%** of a vehicle on their own, justifying a dedicated trip.

3.  **Route Consolidation:**
    * Unlike the previous scenario where vehicles traveled with partial loads (e.g., 48 kg of 60 = 80%), here more merchandise is moved (all customers) with almost total use of available space on each trip.
    * This demonstrates that a **33% increase in capacity** (from 60 to 80) not only allows serving larger customers but drastically improves the combinatorial efficiency of the rest of the fleet.